- NLP w/ RNN: Generate play
- Florent Dufour
- 2016 - 2022

In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
import numpy as np
import os

In [2]:
# Dataset: R&J - Shakespeare
path_to_file = tf.keras.utils.get_file("Shakespeare.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1130496/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding="utf-8")
print(len(text)) # 1115394 characters
print(text[:500])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
# Pre-processing: Encode as integers

vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]
First Citizen


In [5]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [6]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [7]:
# Building the model

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Loss function

for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

(64, 100, 65) # (batch_size, sequence_length, vocab_size)
64
tf.Tensor(
[[[-2.80629168e-03 -8.31973739e-06 -1.36144727e-03 ... -8.85163317e-04
   -2.20908760e-03 -3.48567846e-03]
  [ 5.64133609e-03  1.81904261e-03 -4.98019042e-04 ... -4.41337004e-04
   -3.07981251e-03 -1.13467372e-03]
  [ 2.60318490e-03  7.64396507e-03  1.73502672e-03 ...  2.05316721e-03
    1.03418413e-03 -7.25862291e-03]
  ...
  [-1.09323002e-02 -2.83308653e-03 -3.98997799e-04 ...  2.21677800e-03
    1.38025684e-02 -3.89999268e-03]
  [-1.17545109e-02 -4.93227551e-03 -1.40566402e-03 ... -3.60211590e-04
    1.20199565e-02 -4.21045348e-03]
  [-2.27116933e-03 -2.70372862e-03 -6.52246526e-04 ... -1.23163802e-04
    8.45741853e-03 -6.70667039e-04]]

 [[-1.36682508e-03  9.64666484e-04 -3.46181914e-03 ... -2.97155185e-03
    4.88506351e-03 -6.50231552e-04]
  [ 1.95196015e-03 -3.64710065e-03  2.29514018e-03 ... -4.35468461e-03
    8.56598374e-03 -2.60676187e-03]
  [ 2.54668994e-04  2.51209969e-03  4.08339594e-03 ... -1.140904

In [9]:
# Compiling the model

model.compile(optimizer='adam', loss=loss)

# Checkpoints

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [10]:
# Training

history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Epoch 1/50
172/172 [==============================] - 15s 63ms/step - loss: 2.6806
Epoch 2/50
172/172 [==============================] - 12s 64ms/step - loss: 1.9799
Epoch 3/50
172/172 [==============================] - 13s 65ms/step - loss: 1.7143
Epoch 4/50
172/172 [==============================] - 13s 66ms/step - loss: 1.5678
Epoch 5/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4784
Epoch 6/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4183
Epoch 7/50
172/172 [==============================] - 14s 68ms/step - loss: 1.3746
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3383
Epoch 9/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3062
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2781
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2502
Epoch 12/50
172/172 [==============================] - 13s 69ms/step - loss: 1.2225
E

In [11]:
# Load checkpoints

# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

In [12]:
# Generating text

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [13]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Romeo then said
Romeo then said 'That meet,
How effection now, nor want of death?
O same advice four.

LARTIUS:
O, that's the case is most out of hope, I say.

HASTINGS:
My Lord of Buckingham, now putting over mine eyes,
Men are be no better endured both your hands;
Gives steal thoughts of carries none but toil; how it is your name? I humy welcome;
And, by the airy word! Believe me, love!

VOLUMNIA:
If I do bring thee hence to Friar Lodowick to his captain Christ,
Under whose colours yet I oft it not:
He hath the demedness home again with the entraits of your dear.

Lord:
Those cordial, Duft that, thy taughters, who loss of her,
Upon my cousin; let my brother die,
I fearn us not.

MENENIUS:
O' the treacherous vengeance on offenders' fest
Unto the brother and the canopy for
Your brother's life? or sign from see, since th
